In [ ]:
# Install dependencies if needed
# !pip install fastf1 matplotlib seaborn pandas numpy plotly pillow

In [ ]:
# Core imports
import fastf1
import fastf1.plotting
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.colors import Normalize, LinearSegmentedColormap
import seaborn as sns

# Setup
fastf1.plotting.setup_mpl(mpl_timedelta_support=True, color_scheme='fastf1')

# Enable caching for faster data loading
fastf1.Cache.enable_cache('./cache')

# Set dark theme for TikTok-style visuals
plt.style.use('dark_background')

print(f"FastF1 version: {fastf1.__version__}")
print("✅ Setup complete!")

## 📊 Part 1: Load Sample Data

Let's start by loading data from the 2023 season to understand the data structure.

In [ ]:
# Load 2023 Bahrain GP Qualifying
session = fastf1.get_session(2023, 'Bahrain', 'Q')
session.load()

print(f"Session: {session.event['EventName']}")
print(f"Date: {session.date}")
print(f"Drivers: {session.drivers}")

In [ ]:
# Get the fastest lap for Verstappen
ver_lap = session.laps.pick_drivers('VER').pick_fastest()
print(f"Verstappen fastest lap: {ver_lap['LapTime']}")
print(f"Sectors: S1={ver_lap['Sector1Time']}, S2={ver_lap['Sector2Time']}, S3={ver_lap['Sector3Time']}")

In [ ]:
# Get telemetry data
ver_tel = ver_lap.get_telemetry()
print(f"Telemetry columns: {ver_tel.columns.tolist()}")
print(f"\nTelemetry sample:")
ver_tel.head(10)

## 🎨 Part 2: Speed Trace Visualization

Compare speed traces of the top drivers with team colors.

In [ ]:
# Get fastest laps for top drivers
drivers_to_compare = ['VER', 'LEC', 'HAM', 'PER']

fig, ax = plt.subplots(figsize=(14, 8))

for driver in drivers_to_compare:
    lap = session.laps.pick_drivers(driver).pick_fastest()
    if lap is None:
        continue
    
    telemetry = lap.get_telemetry().add_distance()
    
    # Get team color
    try:
        color = fastf1.plotting.get_driver_color(driver, session)
    except:
        color = '#FFFFFF'
    
    ax.plot(telemetry['Distance'], telemetry['Speed'], 
            label=f"{driver} ({lap['LapTime']})", 
            color=color, linewidth=2)

ax.set_xlabel('Distance (m)', fontsize=12)
ax.set_ylabel('Speed (km/h)', fontsize=12)
ax.set_title('2023 Bahrain GP - Qualifying Speed Comparison', fontsize=16, fontweight='bold')
ax.legend(loc='lower right')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('charts/speed_trace_comparison.png', dpi=200, facecolor='#1E1E1E')
plt.show()

## 🗺️ Part 3: Track Map Visualization

Create a beautiful track map colored by speed.

In [ ]:
# Get Verstappen's fastest lap telemetry
ver_lap = session.laps.pick_drivers('VER').pick_fastest()
ver_tel = ver_lap.get_telemetry()

# Position data
x = ver_tel['X'].values
y = ver_tel['Y'].values
speed = ver_tel['Speed'].values

# Rotate track
circuit_info = session.get_circuit_info()
if circuit_info:
    rotation = circuit_info.rotation
    x_rot = x * np.cos(np.radians(rotation)) - y * np.sin(np.radians(rotation))
    y_rot = x * np.sin(np.radians(rotation)) + y * np.cos(np.radians(rotation))
    x, y = x_rot, y_rot

# Create figure
fig, ax = plt.subplots(figsize=(12, 12))

# Create colored segments
points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

norm = Normalize(vmin=speed.min(), vmax=speed.max())
lc = LineCollection(segments, cmap='RdYlGn', norm=norm, linewidth=4)
lc.set_array(speed)

ax.add_collection(lc)
ax.autoscale()
ax.set_aspect('equal')
ax.axis('off')

# Colorbar
cbar = plt.colorbar(lc, ax=ax, shrink=0.8)
cbar.set_label('Speed (km/h)', fontsize=12)

# Title
ax.set_title(f"Verstappen - Bahrain Track Speed\n{ver_lap['LapTime']}", 
             fontsize=18, fontweight='bold', color='#3671C6')

plt.tight_layout()
plt.savefig('charts/track_speed_map.png', dpi=200, facecolor='#1E1E1E')
plt.show()

## 🆚 Part 4: Head-to-Head Comparison

Compare two drivers on the same track - colored by who's faster.

In [ ]:
# Compare Verstappen vs Leclerc
ver_lap = session.laps.pick_drivers('VER').pick_fastest()
lec_lap = session.laps.pick_drivers('LEC').pick_fastest()

ver_tel = ver_lap.get_telemetry().add_distance()
lec_tel = lec_lap.get_telemetry().add_distance()

# Interpolate to common distance points
max_dist = min(ver_tel['Distance'].max(), lec_tel['Distance'].max())
distances = np.linspace(0, max_dist, 1000)

ver_speed = np.interp(distances, ver_tel['Distance'], ver_tel['Speed'])
lec_speed = np.interp(distances, lec_tel['Distance'], lec_tel['Speed'])
x = np.interp(distances, ver_tel['Distance'], ver_tel['X'])
y = np.interp(distances, ver_tel['Distance'], ver_tel['Y'])

# Rotate
if circuit_info:
    rotation = circuit_info.rotation
    x_rot = x * np.cos(np.radians(rotation)) - y * np.sin(np.radians(rotation))
    y_rot = x * np.sin(np.radians(rotation)) + y * np.cos(np.radians(rotation))
    x, y = x_rot, y_rot

# Speed delta (positive = Verstappen faster)
speed_delta = ver_speed - lec_speed

# Team colors
rb_color = '#3671C6'  # Red Bull
ferrari_color = '#E8002D'  # Ferrari

# Custom colormap
cmap = LinearSegmentedColormap.from_list('team_cmap', [ferrari_color, '#FFFFFF', rb_color])

# Create figure
fig, ax = plt.subplots(figsize=(12, 12))

# Create colored segments
points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

max_diff = max(abs(speed_delta.min()), abs(speed_delta.max()))
norm = Normalize(vmin=-max_diff, vmax=max_diff)

lc = LineCollection(segments, cmap=cmap, norm=norm, linewidth=5)
lc.set_array(speed_delta)

ax.add_collection(lc)
ax.autoscale()
ax.set_aspect('equal')
ax.axis('off')

# Colorbar
cbar = plt.colorbar(lc, ax=ax, shrink=0.8)
cbar.set_label('← LEC faster | VER faster →', fontsize=12)

ax.set_title('Verstappen vs Leclerc - Bahrain 2023', fontsize=18, fontweight='bold')

plt.tight_layout()
plt.savefig('charts/ver_vs_lec_comparison.png', dpi=200, facecolor='#1E1E1E')
plt.show()

## 📈 Part 5: Sector Time Analysis

In [ ]:
# Get sector times for all drivers
drivers_to_compare = ['VER', 'LEC', 'HAM', 'PER', 'SAI', 'RUS']

sector_data = []
for driver in drivers_to_compare:
    lap = session.laps.pick_drivers(driver).pick_fastest()
    if lap is not None and pd.notna(lap['Sector1Time']):
        sector_data.append({
            'Driver': driver,
            'Team': lap['Team'],
            'S1': lap['Sector1Time'].total_seconds(),
            'S2': lap['Sector2Time'].total_seconds(),
            'S3': lap['Sector3Time'].total_seconds(),
            'Total': lap['LapTime'].total_seconds() if pd.notna(lap['LapTime']) else 0
        })

df = pd.DataFrame(sector_data).sort_values('Total')
df

In [ ]:
# Create stacked bar chart
fig, ax = plt.subplots(figsize=(12, 8))

y = np.arange(len(df))
height = 0.6

# Get colors
colors = []
for team in df['Team']:
    try:
        c = fastf1.plotting.get_team_color(team, session)
    except:
        c = '#FFFFFF'
    colors.append(c)

# Plot stacked bars
ax.barh(y, df['S1'], height, label='Sector 1', color=[c + '99' for c in colors], edgecolor='white')
ax.barh(y, df['S2'], height, left=df['S1'], label='Sector 2', color=[c + 'CC' for c in colors], edgecolor='white')
ax.barh(y, df['S3'], height, left=df['S1'] + df['S2'], label='Sector 3', color=colors, edgecolor='white')

ax.set_yticks(y)
ax.set_yticklabels(df['Driver'], fontsize=12)
ax.set_xlabel('Time (seconds)', fontsize=12)
ax.set_title('Sector Time Comparison - Bahrain 2023', fontsize=16, fontweight='bold')
ax.legend(loc='lower right')

# Add lap time labels
for i, (_, row) in enumerate(df.iterrows()):
    ax.text(row['Total'] + 0.1, i, f"{row['Total']:.3f}s", va='center', fontsize=10)

plt.tight_layout()
plt.savefig('charts/sector_comparison.png', dpi=200, facecolor='#1E1E1E')
plt.show()

## 🔬 Part 6: Full Telemetry Comparison Panel

In [ ]:
# Create multi-panel telemetry comparison
ver_lap = session.laps.pick_drivers('VER').pick_fastest()
lec_lap = session.laps.pick_drivers('LEC').pick_fastest()

ver_tel = ver_lap.get_telemetry().add_distance()
lec_tel = lec_lap.get_telemetry().add_distance()

# Get colors
try:
    ver_color = fastf1.plotting.get_driver_color('VER', session)
    lec_color = fastf1.plotting.get_driver_color('LEC', session)
except:
    ver_color, lec_color = '#3671C6', '#E8002D'

fig, axes = plt.subplots(4, 1, figsize=(14, 16), sharex=True)

# Speed
axes[0].plot(ver_tel['Distance'], ver_tel['Speed'], color=ver_color, label='Verstappen')
axes[0].plot(lec_tel['Distance'], lec_tel['Speed'], color=lec_color, label='Leclerc')
axes[0].set_ylabel('Speed (km/h)')
axes[0].legend(loc='upper right')
axes[0].set_title('Telemetry Comparison: Verstappen vs Leclerc', fontsize=16, fontweight='bold')

# Throttle
axes[1].plot(ver_tel['Distance'], ver_tel['Throttle'], color=ver_color)
axes[1].plot(lec_tel['Distance'], lec_tel['Throttle'], color=lec_color)
axes[1].set_ylabel('Throttle (%)')
axes[1].set_ylim(0, 100)

# Brake
axes[2].plot(ver_tel['Distance'], ver_tel['Brake'].astype(int) * 100, color=ver_color)
axes[2].plot(lec_tel['Distance'], lec_tel['Brake'].astype(int) * 100, color=lec_color)
axes[2].set_ylabel('Brake (%)')
axes[2].set_ylim(0, 100)

# Gear
axes[3].plot(ver_tel['Distance'], ver_tel['nGear'], color=ver_color)
axes[3].plot(lec_tel['Distance'], lec_tel['nGear'], color=lec_color)
axes[3].set_ylabel('Gear')
axes[3].set_xlabel('Distance (m)')
axes[3].set_ylim(0, 8)

for ax in axes:
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('charts/telemetry_panel.png', dpi=200, facecolor='#1E1E1E')
plt.show()

## 📊 Part 7: Gap to Leader Analysis

This helps visualize dominance - how much faster was the leader?

In [ ]:
# Calculate gaps to leader
gap_data = []

for driver in session.drivers:
    lap = session.laps.pick_drivers(driver).pick_fastest()
    if lap is not None and pd.notna(lap['LapTime']):
        gap_data.append({
            'Driver': driver,
            'Team': lap['Team'],
            'LapTime': lap['LapTime'].total_seconds()
        })

df = pd.DataFrame(gap_data).sort_values('LapTime')
df['Gap'] = df['LapTime'] - df['LapTime'].min()
df['GapPercent'] = (df['Gap'] / df['LapTime'].min()) * 100

print("Gap to Leader:")
df.head(10)

In [ ]:
# Visualize gaps
fig, ax = plt.subplots(figsize=(12, 8))

top_10 = df.head(10)

colors = []
for team in top_10['Team']:
    try:
        c = fastf1.plotting.get_team_color(team, session)
    except:
        c = '#FFFFFF'
    colors.append(c)

bars = ax.barh(range(len(top_10)), top_10['Gap'], color=colors, edgecolor='white')
ax.set_yticks(range(len(top_10)))
ax.set_yticklabels(top_10['Driver'])
ax.invert_yaxis()  # Fastest at top
ax.set_xlabel('Gap to Leader (seconds)')
ax.set_title('Gap to Pole Position - Bahrain 2023 Qualifying', fontsize=16, fontweight='bold')

# Add gap labels
for i, (_, row) in enumerate(top_10.iterrows()):
    ax.text(row['Gap'] + 0.02, i, f"+{row['Gap']:.3f}s", va='center', fontsize=10)

plt.tight_layout()
plt.savefig('charts/gap_to_pole.png', dpi=200, facecolor='#1E1E1E')
plt.show()

## 🏆 Part 8: Cross-Era Comparison

Now let's compare across different seasons to determine the most dominant car.

In [ ]:
# Define the eras to compare
eras = {
    '2020 - Mercedes W11': {'year': 2020, 'team': 'Mercedes', 'drivers': ['HAM', 'BOT']},
    '2023 - Red Bull RB19': {'year': 2023, 'team': 'Red Bull Racing', 'drivers': ['VER', 'PER']},
    '2025 - McLaren MCL39': {'year': 2025, 'team': 'McLaren', 'drivers': ['NOR', 'PIA']}
}

# Note: 2025 data might not be available yet
# We'll analyze available years

era_results = []

for era_name, config in list(eras.items())[:2]:  # Only 2020 and 2023 for now
    try:
        schedule = fastf1.get_event_schedule(config['year'])
        races = schedule[schedule['EventFormat'] != 'testing']['EventName'].tolist()
        
        # Analyze first race
        first_race = races[0] if races else None
        if first_race:
            session = fastf1.get_session(config['year'], first_race, 'Q')
            session.load()
            
            # Check pole position
            results = session.results.sort_values('Position')
            pole_driver = results.iloc[0]['Abbreviation']
            
            era_results.append({
                'Era': era_name,
                'Year': config['year'],
                'FirstRace': first_race,
                'PoleDriver': pole_driver,
                'TeamPole': pole_driver in config['drivers']
            })
            print(f"✅ {era_name}: {first_race} - Pole: {pole_driver}")
    except Exception as e:
        print(f"❌ {era_name}: {e}")

pd.DataFrame(era_results)

## 📱 Part 9: Export for TikTok

Create TikTok-optimized exports (9:16 aspect ratio).

In [ ]:
from PIL import Image
import os

# Create TikTok-ready directory
os.makedirs('tiktok_ready', exist_ok=True)

def crop_for_tiktok(input_path, output_path):
    """Crop image to 9:16 TikTok format"""
    with Image.open(input_path) as img:
        if img.mode in ('RGBA', 'P'):
            img = img.convert('RGB')
        
        target_ratio = 9 / 16
        current_ratio = img.width / img.height
        
        if current_ratio > target_ratio:
            new_width = int(img.height * target_ratio)
            left = (img.width - new_width) // 2
            img = img.crop((left, 0, left + new_width, img.height))
        else:
            new_height = int(img.width / target_ratio)
            top = (img.height - new_height) // 2
            img = img.crop((0, top, img.width, top + new_height))
        
        img = img.resize((1080, 1920), Image.Resampling.LANCZOS)
        img.save(output_path, quality=95)
    print(f"✅ Saved: {output_path}")

# Convert existing charts
for chart in ['speed_trace_comparison.png', 'track_speed_map.png', 'ver_vs_lec_comparison.png']:
    input_path = f'charts/{chart}'
    if os.path.exists(input_path):
        output_path = f'tiktok_ready/{chart}'
        crop_for_tiktok(input_path, output_path)

## 📝 Summary

This notebook demonstrates how to:

1. **Load F1 data** using FastF1 API
2. **Create speed trace comparisons** with team colors
3. **Visualize track maps** colored by speed or relative performance
4. **Analyze sector times** and gaps to the leader
5. **Compare telemetry** (speed, throttle, brake, gear)
6. **Export for TikTok** in 9:16 format

### Key Findings:
- **Fastest** = Raw lap time performance
- **Dominant** = Gap to competitors + consistency

Use the main.py script for full season analysis!

In [ ]:
print("\n🏁 Analysis complete!")
print("\nNext steps:")
print("1. Run 'python main.py --full' for complete era comparison")
print("2. Check the 'charts/' folder for generated visualizations")
print("3. Check 'tiktok_ready/' for social media optimized exports")